In [2]:
!pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 17.1 MB/s 


In [33]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 13681, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 13681 (delta 350), reused 435 (delta 312), pack-reused 13198
Receiving objects: 100% (13681/13681), 13.22 MiB | 27.91 MiB/s, done.
Resolving deltas: 100% (9411/9411), done.
/content/yolov5


In [59]:
import cv2
from PIL import Image as i
import torch
import torchvision
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.12.1+cu113 (CPU)


In [60]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5/runs/train/exp2/weights/best.pt', force_reload=True)
model.eval()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-10-8 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7053277 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [63]:
# Load a pre-trained version of MobileNetV2
torch_model = model
# Set the model in evaluation mode.
torch_model.eval()

# Trace the model with random data.
example_input = torch.rand(1, 3, 224, 224) 
traced_model = torch.jit.trace(torch_model, example_input)
out = traced_model(example_input)

/root/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:
/root/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


In [64]:
import coremltools as ct

# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.
model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

Converting graph.
Adding op 'model.model.model.0.conv.weight' of type const
Adding op 'model.model.model.0.conv.bias' of type const
Adding op 'model.model.model.1.conv.bias' of type const
Adding op 'model.model.model.1.conv.weight' of type const
Adding op 'model.model.model.2.cv1.conv.bias' of type const
Adding op 'model.model.model.2.cv1.conv.weight' of type const
Adding op 'model.model.model.2.m.0.cv1.conv.bias' of type const
Adding op 'model.model.model.2.m.0.cv1.conv.weight' of type const
Adding op 'model.model.model.2.m.0.cv2.conv.bias' of type const
Adding op 'model.model.model.2.m.0.cv2.conv.weight' of type const
Adding op 'model.model.model.2.cv2.conv.bias' of type const
Adding op 'model.model.model.2.cv2.conv.weight' of type const
Adding op 'model.model.model.2.cv3.conv.bias' of type const
Adding op 'model.model.model.2.cv3.conv.weight' of type const
Adding op 'model.model.model.3.conv.bias' of type const
Adding op 'model.model.model.3.conv.weight' of type const
Adding op 'mod

In [65]:
# Save the converted model.
model.save("newmodel.mlpackage")

In [66]:
# Using image_input in the inputs parameter:
# Convert to Core ML neural network using the Unified Conversion API.
model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

Converting graph.
Adding op 'model.model.model.0.conv.weight' of type const
Adding op 'model.model.model.0.conv.bias' of type const
Adding op 'model.model.model.1.conv.bias' of type const
Adding op 'model.model.model.1.conv.weight' of type const
Adding op 'model.model.model.2.cv1.conv.bias' of type const
Adding op 'model.model.model.2.cv1.conv.weight' of type const
Adding op 'model.model.model.2.m.0.cv1.conv.bias' of type const
Adding op 'model.model.model.2.m.0.cv1.conv.weight' of type const
Adding op 'model.model.model.2.m.0.cv2.conv.bias' of type const
Adding op 'model.model.model.2.m.0.cv2.conv.weight' of type const
Adding op 'model.model.model.2.cv2.conv.bias' of type const
Adding op 'model.model.model.2.cv2.conv.weight' of type const
Adding op 'model.model.model.2.cv3.conv.bias' of type const
Adding op 'model.model.model.2.cv3.conv.weight' of type const
Adding op 'model.model.model.3.conv.bias' of type const
Adding op 'model.model.model.3.conv.weight' of type const
Adding op 'mod

In [70]:
# Save the converted model.
model.save("model2.mlmodel")